In [1]:
!pip install -q transformers torch


In [2]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer


In [3]:
gpt2_name = "gpt2"

gpt2_tokenizer = GPT2Tokenizer.from_pretrained(gpt2_name)
gpt2_model = GPT2LMHeadModel.from_pretrained(gpt2_name)


gpt2_tokenizer.pad_token = gpt2_tokenizer.eos_token

gpt2_model.eval()


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [4]:
dialogpt_name = "microsoft/DialoGPT-small"

dialogpt_tokenizer = GPT2Tokenizer.from_pretrained(dialogpt_name)
dialogpt_model = GPT2LMHeadModel.from_pretrained(dialogpt_name)


dialogpt_tokenizer.pad_token = dialogpt_tokenizer.eos_token

dialogpt_model.eval()


tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/641 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/351M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [5]:
def generate(model, tokenizer, prompt, max_new_tokens=80):
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        padding=True
    )

    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            repetition_penalty=1.2,
            pad_token_id=tokenizer.eos_token_id
        )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [6]:
gpt2_conversation = """The following is a conversation with an AI assistant.

"""

def chat_gpt2(user_input):
    global gpt2_conversation

    gpt2_conversation += f"User: {user_input}\nAssistant:"
    response = generate(gpt2_model, gpt2_tokenizer, gpt2_conversation)

    assistant_reply = response[len(gpt2_conversation):]
    assistant_reply = assistant_reply.split("User:")[0]

    gpt2_conversation += assistant_reply.strip() + "\n"
    return assistant_reply.strip()


In [10]:
dialogpt_conversation = """The following is a conversation with an AI assistant.

"""

def chat_dialogpt(user_input):
    global dialogpt_conversation

    dialogpt_conversation += f"User: {user_input}\nAssistant:"
    response = generate(dialogpt_model, dialogpt_tokenizer, dialogpt_conversation)

    assistant_reply = response[len(dialogpt_conversation):]
    assistant_reply = assistant_reply.split("User:")[0]

    dialogpt_conversation += assistant_reply.strip() + "\n"
    return assistant_reply.strip()


In [11]:
question = "What is Artificial Intelligence?"

print("GPT-2:")
print(chat_gpt2(question))

print("\nDialoGPT:")
print(chat_dialogpt(question))


GPT-2:
If anyone has any suggestions please let us know :)

DialoGPT:
You're not the only one.


In [12]:
while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        break

    print("\nGPT-2:", chat_gpt2(user_input))
    print("DialoGPT:", chat_dialogpt(user_input))


You:  What is Artificial Intelligence



GPT-2: Actions speak louder then Words Speak Less ! This will allow your voice to communicate clearly without having "words" being added into dialogue The language may differ slightly depending upon who uses speech recognition software such Google Translate(!) etc.. But everyone needs equal rights!! Your human nature should not dictate anything whatsoever; In fact using Speech Recognition Technology helps even less under certain circumstances due its ability only improve communication
DialoGPT: You arent alone


You:  exit
